loading package and seurat object

In [ ]:
library(ggplot2)
library(cowplot)
library(dplyr)
library(SCPA)
library(tidyverse)
library(msigdbr)
library(tibble)
library(AnnotationHub)
library(patchwork)
library(Seurat)
library(rliger)
library(SeuratData)
library(magrittr)
library(SeuratWrappers)
library(viridis)
library(tidyr)
library(magrittr)
library(RColorBrewer)

In [ ]:
custom_colors <- list()

colors_dutch <- c("#16a085", "#2980b9", "#4834d4", "#f39c12", "#c0392b",
                  "#130f40", "#f78fb3", "#65d6ce", "#d1c145", "#2ecc71",
                  "#d35400", "#c44569", '#ff7f50', '#706fd3', '#f9ca24',
                  '#34ace0', '#33d9b2', '#2c2c54', '#be2edd')

colors_spanish <- c(
  '#40407a','#706fd3','#f7f1e3','#34ace0','#33d9b2',
  '#2c2c54','#474787','#aaa69d','#227093','#218c74',
  '#ff5252','#ff793f','#d1ccc0','#ffb142','#ffda79',
  '#b33939','#cd6133','#84817a','#cc8e35','#ccae62'
)

custom_colors$discrete <- c(colors_dutch, colors_spanish)

custom_colors$cell_cycle <- setNames(
  c('#45aaf2', '#f1c40f', '#e74c3c', '#7f8c8d'),
  c('G1',      'S',       'G2M',     '-')
)

In [ ]:
options(repr.plot.width = 18, repr.plot.height = 17, repr.plot.pointsize = 24)

In [2]:
microglia_subcuster_IEG <- readRDS("/home/adufour/work/rds_storage/microglia/microglia_IEG.rds")

tb1 <- msigdbr("Mus musculus", category = "C2", subcategory = "CP:KEGG")
tb2 <- msigdbr("Mus musculus", category = "C5", subcategory = "GO:BP")
tb3 <- msigdbr("Mus musculus", category = "H")

pathways <- rbind(tb1, tb2, tb3) %>% format_pathways()

cell_types <- unique(microglia_subcuster_IEG@meta.data$seurat_clusters)

scpa_out <- list()

The legacy packages maptools, rgdal, and rgeos, underpinning this package
will retire shortly. Please refer to R-spatial evolution reports on
https://r-spatial.org/r/2023/05/15/evolution4.html for details.
This package is now running under evolution status 0 



In [3]:
microglia_subcuster_IEG@meta.data$cluster_time <- paste0(microglia_subcuster_IEG@meta.data$PND, "_", microglia_subcuster_IEG@meta.data$seurat_clusters)

In [4]:
cell_types <- unique(microglia_subcuster_IEG@meta.data$seurat_clusters)

In [8]:
microglia_subcuster_IEG.split <- Seurat::SplitObject(microglia_subcuster_IEG, split.by = "STIM")

In [9]:
scpa_out <- list()
for (i in cell_types) {
  
  condition_pbs <- seurat_extract(microglia_subcuster_IEG.split$pbs, 
                            meta1 = "seurat_clusters", value_meta1 = i)
  
  condition_il1 <- seurat_extract(microglia_subcuster_IEG.split$il1, 
                          meta1 = "seurat_clusters", value_meta1 = i)
  
  print(paste("comparing", i))
  scpa_out[[i]] <- compare_pathways(list(condition_pbs, condition_il1), pathways, parallel = TRUE, core = 4)

# For faster analysis with parallel processing, use 'parallel = TRUE' and 'cores = x' arguments
  
}

Extracting cells where seurat_clusters == 2

Extracting data from the RNA assay

Extracting cells where seurat_clusters == 2

Extracting data from the RNA assay



[1] "comparing 2"


Processing in parallel using 4 cores


Cell numbers in population 1 = 1150

Cell numbers in population 2 = 2726

- If greater than 500 cells, these populations will be downsampled


Excluding 3642 pathway(s) based on min/max genes parameter: GOBP_10_FORMYLTETRAHYDROFOLATE_METABOLIC_PROCESS, GOBP_2FE_2S_CLUSTER_ASSEMBLY, GOBP_3_PHOSPHOADENOSINE_5_PHOSPHOSULFATE_BIOSYNTHETIC_PROCESS, GOBP_5S_CLASS_RRNA_TRANSCRIPTION_BY_RNA_POLYMERASE_III, GOBP_5_PHOSPHORIBOSE_1_DIPHOSPHATE_METABOLIC_PROCESS...


Calculating pathway fold changes...


Performing a two-sample analysis with SCPA...

Extracting cells where seurat_clusters == 1

Extracting data from the RNA assay

Extracting cells where seurat_clusters == 1

Extracting data from the RNA assay



[1] "comparing 1"


Processing in parallel using 4 cores


Cell numbers in population 1 = 2137

Cell numbers in population 2 = 2033

- If greater than 500 cells, these populations will be downsampled


Excluding 3642 pathway(s) based on min/max genes parameter: GOBP_10_FORMYLTETRAHYDROFOLATE_METABOLIC_PROCESS, GOBP_2FE_2S_CLUSTER_ASSEMBLY, GOBP_3_PHOSPHOADENOSINE_5_PHOSPHOSULFATE_BIOSYNTHETIC_PROCESS, GOBP_5S_CLASS_RRNA_TRANSCRIPTION_BY_RNA_POLYMERASE_III, GOBP_5_PHOSPHORIBOSE_1_DIPHOSPHATE_METABOLIC_PROCESS...


Calculating pathway fold changes...


Performing a two-sample analysis with SCPA...

Extracting cells where seurat_clusters == 0

Extracting data from the RNA assay

Extracting cells where seurat_clusters == 0

Extracting data from the RNA assay



[1] "comparing 0"


Processing in parallel using 4 cores


Cell numbers in population 1 = 2600

Cell numbers in population 2 = 4066

- If greater than 500 cells, these populations will be downsampled


Excluding 3642 pathway(s) based on min/max genes parameter: GOBP_10_FORMYLTETRAHYDROFOLATE_METABOLIC_PROCESS, GOBP_2FE_2S_CLUSTER_ASSEMBLY, GOBP_3_PHOSPHOADENOSINE_5_PHOSPHOSULFATE_BIOSYNTHETIC_PROCESS, GOBP_5S_CLASS_RRNA_TRANSCRIPTION_BY_RNA_POLYMERASE_III, GOBP_5_PHOSPHORIBOSE_1_DIPHOSPHATE_METABOLIC_PROCESS...


Calculating pathway fold changes...


Performing a two-sample analysis with SCPA...



In [16]:
for (i in cell_types) {
  scpa_out[[i]] <- scpa_out[[i]] %>%
    select(Pathway, qval) %>%
    colnames(c("Pathway", paste(i, "qval", sep = "_")))

# For faster analysis with parallel processing, use 'parallel = TRUE' and 'cores = x' arguments
}

ERROR: Error in is.data.frame(x) && do.NULL: type 'y' incorrect dans 'x && y'


In [19]:
scpa_out <- scpa_out %>% 
  reduce(full_join, by = "Pathway")

In [20]:
write.csv(scpa_out, "/home/adufour/work/table/microglia_scpa_subcluster.csv")

In [13]:
scpa_out <- scpa_out[rowSums(scpa_out[])>0,]

In [16]:
scpa_out <- scpa_out[grep("HALLMARK", rownames(scpa_out)),]

In [37]:
rownames(scpa_out) <- sub('HALLMARK_', '', rownames(scpa_out))
rownames(scpa_out) <- sub('_', ' ', rownames(scpa_out))

In [30]:
colnames(scpa_out) <- c("Cluster 2", "Cluster 1", "Cluster 0")

In [31]:
column_order <- c("Cluster 0", "Cluster 1", "Cluster 2")
mat <- t(scale(t(scpa_out[,column_order]), center = T, scale=T))
hb <- HeatmapAnnotation(foo = anno_block(gp = gpar(fill = c("#00a085", "#1a7fb6", "#4a2ed0")),
                                                   labels = c('0', '1', '2'),
                                         labels_gp = gpar(col = c("black", "white", "white"), fontsize = 18)))
col_fun <- colorRamp2(seq(min(mat), max(mat), length = 100), colorRampPalette(rev(brewer.pal(n = 11, name = "RdYlBu")))(100))
lgd <- list(title = "Q-value", legend_height = unit(6, "cm"), grid_width = unit(1, "cm"), labels_gp = gpar(fontsize = 24), title_gp = gpar(fontsize = 24), title_position = "leftcenter-rot")
split = rep(1:8, each = 3)

In [40]:
options(repr.plot.width=11, repr.plot.height=10) # To set the figure size in Jupyter
png('/home/adufour/work/notebook/plots/Microglia/microglia_hallmark_IEG.png',width=11,height=10,units="in",res=300)
hm <- draw(ComplexHeatmap::Heatmap(mat, 
                                   col = col_fun, 
                                   cluster_columns = FALSE, 
                                   show_row_names=TRUE,
                                   column_names_gp=grid::gpar(fontsize=24),
                                   row_names_gp=grid::gpar(fontsize=24),
                                   column_gap = unit(3, "mm"), 
                                   column_title = NULL, 
                                   heatmap_legend_param = lgd), 
           padding = unit(c(2, 2, 2, 120), "mm"),
           heatmap_legend_side = "left")
dev.off()

png 
  2